[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [16]:
# Import pandas, numpy
import numpy as np
import pandas as pd

In [17]:
# Read dataset with extra params sep='\t', encoding="latin-1"
sentiment = pd.read_csv("./labeledTrainData.tsv", sep='\t', encoding='latin-1')
sentiment.sample(5)

,id,sentiment,review
17414,7687_1,0,I started to watch this movie with high expect...
24475,8166_10,1,"This movie is very great! The acting is fine, ..."
19211,2635_7,1,"My Take: A funny take on THE LION KING, posing..."
18768,6308_8,1,"Silly, hilarious, tragic, sad, inevitable.<br ..."
21729,2582_8,1,"In the aftermath of Watergate, a number of con..."


## 2. Preprocessing

In [18]:
# stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\HARRY
[nltk_data]     LE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text
#testing function
print(preprocessor(':) : - ? https://lekhanh.org :)'))

 https lekhanh org  :) :)


In [20]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()
#split a text into list of words
def tokenizer_porter(text):
    return text.split()
#Test function
print (tokenizer_porter("Elizabeth Harry Potter Ronal Bilius Weasley .web"))

['Elizabeth', 'Harry', 'Potter', 'Ronal', 'Bilius', 'Weasley', '.web']


In [21]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split
X = sentiment['review']
y = sentiment['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [28]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#import sklearn.preprocessing, sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=115))])
clf.fit(X_train, y_train)

C:\Users\HARRY LE\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x0000016AE5387F78>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'mysel...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x0000016AEE5F33A8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
      

## 4. Evaluate Model

In [29]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Using Test dataset to evaluate model
prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction))

# classification_report
print(classification_report(y_test, prediction))

# confusion matrix
print(confusion_matrix(y_test, prediction))

0.8976
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      2487
           1       0.89      0.91      0.90      2513

    accuracy                           0.90      5000
   macro avg       0.90      0.90      0.90      5000
weighted avg       0.90      0.90      0.90      5000

[[2195  292]
 [ 220 2293]]


## 5. Export Model 

In [31]:
# Using pickle to export our trained model
import pickle
import os

In [33]:
pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)

In [36]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X, y)

C:\Users\HARRY LE\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\HARRY LE\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARRY LE\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\HARRY LE\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.89260859, 0.88480922, 0.88850216])